# Quantum Machine Learning - Quantum Support Vector Machine (QSVM)

Quantum Support Vector Machine is a quantum algorithm that enhances classical machine learning by leveraging quantum feature spaces. This notebook explores QSVM implementation and its potential advantages in classification tasks.

In [ ]:
# Import required libraries
import time

import numpy as np

from ariadne import explain_routing, simulate
from ariadne.algorithms import QSVM, AlgorithmParameters

## Understanding Quantum Support Vector Machine

QSVM enhances classical SVM by using quantum kernels:

**Classical SVM:** $f(x) = \text{sign}(\sum_i \alpha_i y_i K(x_i, x) + b)$
**Quantum enhancement:** $K(x_i, x_j) = |\langle\phi(x_i)|\phi(x_j)\rangle|^2$

where $|\phi(x)\rangle$ is the quantum feature map encoding classical data.

Key components:
1. **Feature Map**: Encodes classical data into quantum states
2. **Kernel Evaluation**: Computes inner products in high-dimensional feature space
3. **Classical Optimization**: Trains SVM on quantum kernel matrix
4. **Classification**: Uses quantum kernel for new data points

## Creating QSVM Circuits with Different Configurations

In [ ]:
# Create QSVM circuits with different configurations
qsvm_configs = [
    {"n_qubits": 2, "name": "QSVM-2 (Basic)", "custom_params": {"use_feature_map": True, "use_variational": False}},
    {"n_qubits": 3, "name": "QSVM-3 (Extended)", "custom_params": {"use_feature_map": True, "use_variational": False}},
    {
        "n_qubits": 4,
        "name": "QSVM-4 (Variational)",
        "custom_params": {"use_feature_map": True, "use_variational": True},
    },
    {
        "n_qubits": 4,
        "name": "QSVM-4 (Feature Map Only)",
        "custom_params": {"use_feature_map": True, "use_variational": False},
    },
]

qsvm_circuits = {}

for config in qsvm_configs:
    params = AlgorithmParameters(n_qubits=config["n_qubits"], custom_params=config["custom_params"])

    qsvm = QSVM(params)
    circuit = qsvm.create_circuit()
    qsvm_circuits[config["name"]] = circuit

    print(f"{config['name']}:")
    print(f"  Qubits: {circuit.num_qubits}")
    print(f"  Depth: {circuit.depth()}")
    print(f"  Gate counts: {circuit.count_ops()}")
    print(f"  Feature map: {config['custom_params']['use_feature_map']}")
    print(f"  Variational: {config['custom_params']['use_variational']}")
    print()

## Visualizing QSVM Circuit Structure

In [ ]:
# Display the 4-qubit QSVM circuit with feature map
print("4-qubit QSVM Circuit (with feature map):")
print(qsvm_circuits["QSVM-4 (Feature Map Only)"].draw(output="text"))

## Testing QSVM Across Different Backends

In [ ]:
# Test 4-qubit QSVM across different backends
test_circuit = qsvm_circuits["QSVM-4 (Feature Map Only)"]
backends = ["stim", "qiskit", "mps", "tensor_network"]
results = {}

print("Testing QSVM-4 across backends:")
print("=" * 50)

for backend in backends:
    try:
        start_time = time.time()
        result = simulate(test_circuit, shots=1000, backend=backend)
        end_time = time.time()

        results[backend] = {
            "time": end_time - start_time,
            "counts": result.counts,
            "backend_used": result.backend_used.value,
        }

        print(f"{backend:15} | {end_time - start_time:8.4f}s | {result.backend_used.value}")

    except Exception as e:
        print(f"{backend:15} | FAILED - {str(e)[:30]}")
        results[backend] = {"error": str(e)}

## Analyzing QSVM Measurement Distributions

In [ ]:
# Analyze the measurement outcomes for QSVM
print("\nQSVM Measurement Distribution Analysis:")
print("=" * 50)

for backend, data in results.items():
    if "error" not in data:
        counts = data["counts"]
        total_shots = sum(counts.values())

        print(f"\n{backend.upper()} Results:")

        # Calculate distribution statistics
        num_outcomes = len(counts)
        max_count = max(counts.values())
        min_count = min(counts.values())

        # Calculate entropy (measure of distribution spread)
        entropy = 0.0
        for count in counts.values():
            if count > 0:
                p = count / total_shots
                entropy -= p * np.log2(p)

        print(f"  Number of outcomes: {num_outcomes}")
        print(f"  Entropy: {entropy:.3f} bits")
        print(f"  Max count: {max_count} ({max_count / total_shots:.3f})")
        print(f"  Min count: {min_count} ({min_count / total_shots:.3f})")

        # Show top 5 most frequent outcomes
        sorted_counts = sorted(counts.items(), key=lambda x: x[1], reverse=True)
        print("  Top 5 outcomes:")
        for _i, (outcome, count) in enumerate(sorted_counts[:5]):
            prob = count / total_shots
            print(f"    |{outcome}⟩: {count:4d} ({prob:.3f})")

## Feature Map Analysis

In [ ]:
# Compare QSVM with and without feature maps
print("Feature Map Impact Analysis:")
print("=" * 40)

# Create circuits with different data points
data_points = [
    [0.1, 0.2, 0.3, 0.4],  # Small values
    [0.5, 0.5, 0.5, 0.5],  # Medium values
    [0.9, 0.8, 0.7, 0.6],  # Large values
]

feature_map_results = {}

for i, data_point in enumerate(data_points):
    print(f"\nData Point {i + 1}: {data_point}")

    # With feature map
    params_with_fm = AlgorithmParameters(n_qubits=4, custom_params={"use_feature_map": True, "data_point": data_point})

    qsvm_with_fm = QSVM(params_with_fm)
    circuit_with_fm = qsvm_with_fm.create_circuit()

    try:
        result_with_fm = simulate(circuit_with_fm, shots=1000, backend="qiskit")
        counts_with_fm = result_with_fm.counts

        # Calculate entropy
        total_shots = sum(counts_with_fm.values())
        entropy = 0.0
        for count in counts_with_fm.values():
            if count > 0:
                p = count / total_shots
                entropy -= p * np.log2(p)

        print(f"  With feature map: entropy = {entropy:.3f}")

        feature_map_results[f"point_{i + 1}_with_fm"] = {"entropy": entropy, "counts": counts_with_fm}

    except Exception as e:
        print(f"  With feature map: FAILED - {str(e)[:30]}")

    # Without feature map (basic circuit)
    params_without_fm = AlgorithmParameters(
        n_qubits=4, custom_params={"use_feature_map": False, "data_point": data_point}
    )

    qsvm_without_fm = QSVM(params_without_fm)
    circuit_without_fm = qsvm_without_fm.create_circuit()

    try:
        result_without_fm = simulate(circuit_without_fm, shots=1000, backend="qiskit")
        counts_without_fm = result_without_fm.counts

        # Calculate entropy
        total_shots = sum(counts_without_fm.values())
        entropy = 0.0
        for count in counts_without_fm.values():
            if count > 0:
                p = count / total_shots
                entropy -= p * np.log2(p)

        print(f"  Without feature map: entropy = {entropy:.3f}")

        feature_map_results[f"point_{i + 1}_without_fm"] = {"entropy": entropy, "counts": counts_without_fm}

    except Exception as e:
        print(f"  Without feature map: FAILED - {str(e)[:30]}")

## Scaling Analysis: QSVM Performance with Different Qubit Counts

In [ ]:
# Test scaling behavior of QSVM
scaling_results = {}
test_sizes = [2, 3, 4]  # Different numbers of qubits
test_backend = "qiskit"  # Use reliable backend for scaling test

print(f"QSVM Scaling Analysis (backend: {test_backend}):")
print("=" * 60)
print(f"{'Qubits':<6} | {'Depth':<6} | {'Time (s)':<9} | {'Entropy':<9} | {'Outcomes':<9}")
print("-" * 50)

for n_qubits in test_sizes:
    params = AlgorithmParameters(n_qubits=n_qubits, custom_params={"use_feature_map": True})

    qsvm = QSVM(params)
    circuit = qsvm.create_circuit()

    try:
        start_time = time.time()
        result = simulate(circuit, shots=1000, backend=test_backend)
        end_time = time.time()

        # Calculate entropy
        counts = result.counts
        total_shots = sum(counts.values())
        entropy = 0.0
        for count in counts.values():
            if count > 0:
                p = count / total_shots
                entropy -= p * np.log2(p)

        print(
            f"{n_qubits:<6} | {circuit.depth():<6} | {end_time - start_time:<9.4f} | {entropy:<9.3f} | {len(counts):<9}"
        )

        scaling_results[n_qubits] = {
            "time": end_time - start_time,
            "depth": circuit.depth(),
            "entropy": entropy,
            "outcomes": len(counts),
        }

    except Exception as e:
        print(f"{n_qubits:<6} | FAILED - {str(e)[:20]}")

## QSVM Circuit Analysis

In [ ]:
# Analyze circuit properties for different QSVM configurations
print("\nQSVM Circuit Properties:")
print("=" * 30)

for config_name, _circuit in qsvm_circuits.items():
    # Get the corresponding parameters
    config = next(c for c in qsvm_configs if c["name"] == config_name)

    params = AlgorithmParameters(n_qubits=config["n_qubits"], custom_params=config["custom_params"])

    qsvm = QSVM(params)
    analysis = qsvm.analyze_circuit_properties()

    print(f"\n{config_name}:")
    print(f"  Qubits: {analysis['n_qubits']}")
    print(f"  Depth: {analysis['depth']}")
    print(f"  Total gates: {analysis['size']}")
    print(f"  Two-qubit gates: {analysis['two_qubit_gates']}")
    print(f"  Entanglement heuristic: {analysis['entanglement_heuristic']:.2f}")
    print(f"  Gate types: {list(analysis['gate_counts'].keys())}")

## Understanding Quantum Feature Maps

In [ ]:
# Explain the ZZ feature map used in QSVM
print("Quantum Feature Map (ZZ Feature Map):")
print("=" * 40)
print()
print("Encoding process:")
print("1. Apply Hadamard gates: |0⟩ → (|0⟩ + |1⟩)/√2")
print("2. Apply Z rotations: RZ(2xi) based on data xi")
print("3. Apply entangling ZZ rotations: RZZ(2xi xj) between qubits")
print()
print("Mathematical form:")
print("|φ(x)⟩ = ∏_i H_i RZ(2xi)_i ∏_i<j RZZ(2xi xj)_ij |0⟩^⊗n")
print()
print("Kernel function:")
print("K(x, x') = |⟨φ(x)|φ(x')⟩|^2")
print()
print("Advantages:")
print("- Maps to high-dimensional Hilbert space")
print("- Creates entanglement based on data correlations")
print("- Potentially captures complex patterns in data")
print("- Quantum advantage for certain data distributions")

## Educational Content: QSVM Mathematical Background

In [ ]:
# Get educational content about QSVM
params = AlgorithmParameters(n_qubits=4, custom_params={"use_feature_map": True})

qsvm = QSVM(params)
educational_content = qsvm.get_educational_content()

print("=== QSVM Mathematical Background ===")
print(educational_content["mathematical_background"])

print("\n=== Implementation Notes ===")
print(educational_content["implementation_notes"])

print("\n=== Applications ===")
print(educational_content["applications"])

## Routing Analysis for QSVM

In [ ]:
# Analyze how Ariadne routes QSVM circuits
print("Routing Analysis for QSVM-4:")
print("=" * 30)

explanation = explain_routing(test_circuit)
print(explanation)

## Key Takeaways

1. **Quantum Enhancement**: QSVM uses quantum feature spaces to potentially improve classification
2. **Feature Maps**: Critical for encoding classical data into quantum states
3. **Kernel Evaluation**: Quantum circuits compute inner products in high-dimensional spaces
4. **Hybrid Approach**: Combines quantum kernel evaluation with classical optimization
5. **Variational Options**: Trainable feature maps can be optimized for specific datasets

Quantum Machine Learning represents an exciting frontier where quantum computing meets artificial intelligence. While still in early stages, algorithms like QSVM demonstrate the potential for quantum advantage in specific machine learning tasks, particularly those involving high-dimensional feature spaces and complex data patterns.